In [1]:
# -*- coding: utf-8 -*-
"""
(2024+2025) 동일 '세부분류' 전체 리스트업 + 미개막(아직 안열림) 표시 스크립트
- 로드: ./csv/{2024.csv, 2025.csv} (+ /mnt/data 경로도 시도)
- 입력 축제명의 '세부분류' 라벨을 데이터셋에서 탐지
- 동일 '세부분류'에 속하는 모든 축제를 날짜 필터 없이 나열
- 시작일 > 오늘 인 경우, 축제명 뒤에 '(아직 안열림)' 자동 표시
- 결과: 콘솔 요약 + CSV 저장(동일세부분류_전체목록_표시포함.csv)
"""

import os
import re
import json
from pathlib import Path
from difflib import get_close_matches
from typing import Tuple, Optional, List

import pandas as pd


# ------------------------------------------------------------
# 설정
# ------------------------------------------------------------
CANDIDATE_FILES = [
    "./csv/2024.csv",
    "./csv/2025.csv",
    "/mnt/data/2024.csv",
    "/mnt/data/2025.csv",
]

# 컬럼 동의어 맵
COL_SYNONYMS = {
    "연번": ["연번", "번호", "id", "ID"],
    "광역": ["광역자치단체명","광역","시도","광역시도","광역명"],
    "기초": ["기초자치단체명","기초","시군구","자치구","기초명"],
    "축제명": ["축제명","행사명","이벤트명","명칭","타이틀"],
    "시작일": ["시작일","start","시작","start_date","시작일자","개막일"],
    "종료일": ["종료일","end","종료","end_date","종료일자","폐막일"],
    "대분류": ["대분류","분류대","유형대","축제유형(대)","축제유형_대","분류_대","대분류명"],
    "중분류": ["중분류","분류중","유형중","축제유형(중)","축제유형_중","분류_중","중분류명"],
    "세부분류": ["세부분류","세부유형","소분류","세부분류명","유형세부"],
    # 구버전 단일 분류열 (예: '문화예술-음악·공연' 같은 한 줄짜리)
    "구_축제유형": ["축제유형","축제 유형"],
}


# ------------------------------------------------------------
# 유틸
# ------------------------------------------------------------
def read_csv_any(path: str) -> pd.DataFrame:
    for enc in ["utf-8-sig", "cp949", "utf-8", "euc-kr"]:
        try:
            return pd.read_csv(path, encoding=enc, dtype=str)
        except Exception:
            pass
    return pd.read_csv(path, dtype=str)


def _pick_first_exist(candidates: List[str], df_cols: List[str]) -> Optional[str]:
    s = set(df_cols)
    for c in candidates:
        if c in s:
            return c
    return None


def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """컬럼명을 표준화하고, '세부분류'가 없으면 '구_축제유형' 또는 '대-중'으로 보완."""
    cols = [str(c).strip() for c in df.columns]
    mapping = {}
    for std, syns in COL_SYNONYMS.items():
        found = _pick_first_exist(syns, cols)
        if found:
            mapping[found] = std
    out = df.rename(columns=mapping).copy()

    # 필수 컬럼 기본 생성
    for need in ["연번","광역","기초","축제명","시작일","종료일","대분류","중분류","세부분류","구_축제유형"]:
        if need not in out.columns:
            out[need] = ""

    # 세부분류 보완 로직
    has_fine = out["세부분류"].astype(str).str.strip().ne("").any()
    if not has_fine and out["구_축제유형"].astype(str).str.strip().ne("").any():
        # 단일열(축제유형) → 세부분류로 사용
        out["세부분류"] = out["구_축제유형"].astype(str).str.strip()

    # 그래도 세부분류가 비어 있으면 '대-중'을 합쳐 세부처럼 사용
    still_empty_mask = out["세부분류"].astype(str).str.strip().eq("")
    if still_empty_mask.any():
        combo = (
            out["대분류"].astype(str).str.strip()
            + "-"
            + out["중분류"].astype(str).str.strip()
        ).str.strip("-")
        out.loc[still_empty_mask, "세부분류"] = combo

    return out


def load_all() -> pd.DataFrame:
    frames = []
    for p in CANDIDATE_FILES:
        if Path(p).exists():
            df = read_csv_any(p).fillna("")
            df["_출처파일"] = Path(p).name
            # 파일명에 연도가 있으면 사용
            year = ""
            if "2024" in df["_출처파일"].iloc[0]:
                year = "2024"
            elif "2025" in df["_출처파일"].iloc[0]:
                year = "2025"
            df["_연도(파일)"] = year
            frames.append(standardize_columns(df))
    if not frames:
        raise FileNotFoundError("CSV(2024.csv, 2025.csv)를 찾지 못했습니다.")
    out = pd.concat(frames, ignore_index=True)
    # 중복 제거(보수적): 핵심 필드 기준
    out = out.drop_duplicates(
        subset=["광역","기초","축제명","세부분류","시작일","종료일"],
        keep="first"
    ).reset_index(drop=True)
    return out


def norm(s: str) -> str:
    if s is None:
        return ""
    return re.sub(r"\s+", "", str(s)).strip()


def find_row_by_name(df: pd.DataFrame, name: str) -> Optional[pd.Series]:
    """1) 정확일치 → 2) 공백무시 포함검색 → 3) 유사도"""
    exact = df[df["축제명"].astype(str).str.strip() == name.strip()]
    if not exact.empty:
        return exact.iloc[0]

    qn = norm(name)
    cand = df[df["축제명"].astype(str).apply(lambda x: (norm(x) in qn) or (qn in norm(x)))]
    if not cand.empty:
        return cand.iloc[0]

    pool = df["축제명"].astype(str).tolist()
    hit = get_close_matches(name, pool, n=1, cutoff=0.80)
    if hit:
        return df[df["축제명"] == hit[0]].iloc[0]

    return None


# ------------------------------------------------------------
# 핵심 함수
# ------------------------------------------------------------
def list_all_same_fine(query_name: str, include_self: bool = True) -> Tuple[pd.DataFrame, dict]:
    """
    입력 축제명과 동일한 '세부분류' 전체 목록을 반환하고, 시작일이 미래면 (아직 안열림) 표기.
    반환: (DataFrame, meta_dict)
    """
    df = load_all()
    row = find_row_by_name(df, query_name)
    if row is None:
        raise RuntimeError(f"[오류] 입력 축제명을 찾지 못했습니다: {query_name}")

    target_fine = str(row.get("세부분류", "")).strip()
    if target_fine == "":
        raise RuntimeError("[오류] 이 축제의 '세부분류' 라벨을 찾지 못했습니다.")

    same = df[df["세부분류"].astype(str).str.strip() == target_fine].copy()

    if not include_self:
        same = same[same["축제명"].astype(str).str.strip() != str(row["축제명"]).strip()]

    # (아직 안열림) 표시
    today = pd.Timestamp.today().normalize()
    same["_시작일_ts"] = pd.to_datetime(same["시작일"], errors="coerce")

    def annotate(r):
        nm = str(r["축제명"])
        s = r["_시작일_ts"]
        if pd.notna(s) and s > today:
            return f"{nm} (아직 안열림)"
        return nm

    same["표시명"] = same.apply(annotate, axis=1)

    # 연도 컬럼 생성(시작일의 연도 우선, 없으면 파일 연도)
    same["_연도"] = same["_시작일_ts"].dt.year.astype("Int64").astype(str)
    same.loc[same["_연도"].isin(["<NA>","nan","None",""]), "_연도"] = same["_연도(파일)"].fillna("").astype(str)

    # 정렬: 연도 desc → 시작일 asc → 축제명 asc
    same = same.sort_values(by=["_연도","_시작일_ts","축제명"], ascending=[False, True, True])

    # 출력 컬럼
    keep = [c for c in ["연번","광역","기초","축제명","표시명","세부분류","시작일","종료일","_연도","_출처파일"] if c in same.columns]
    same = same[keep].reset_index(drop=True)

    # 요약 메타
    by_year = same.groupby("_연도").size().rename("개수").reset_index()
    meta = {
        "입력축제명": query_name,
        "탐지세부분류": target_fine,
        "연도별개수": dict(zip(by_year["_연도"], by_year["개수"])),
        "총개수": int(len(same))
    }

    # 저장
    out_csv = "동일세부분류_전체목록_표시포함.csv"
    same.to_csv(out_csv, index=False, encoding="utf-8-sig")
    with open("동일세부분류_개요_표시포함.json", "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    # 콘솔 출력(요약)
    print(f"[입력] {query_name}")
    print(f"[세부분류] {target_fine}")
    print(f"[총개수] {meta['총개수']}  |  [연도별] {meta['연도별개수']}")
    print(f"[저장] {out_csv}")

    return same, meta


# ------------------------------------------------------------
# 예시 실행
# ------------------------------------------------------------
if __name__ == "__main__":
    # 예: 담양산타축제 → 동일 세부분류 전체 나열(자기 자신 포함)
    # 다른 축제명을 넣어 실행해도 됩니다.
    FESTIVAL_NAME = "담양산타축제"
    df_list, meta = list_all_same_fine(FESTIVAL_NAME, include_self=True)

    # 필요시: 자기 자신 제외
    # df_list, meta = list_all_same_fine(FESTIVAL_NAME, include_self=False)


[입력] 담양산타축제
[세부분류] 문화예술-크리스마스·산타
[총개수] 6  |  [연도별] {'2024': 2, '2025': 4}
[저장] 동일세부분류_전체목록_표시포함.csv
